In [12]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

# 1. Загрузка данных
data = pd.read_csv('processed_data.csv')  # путь к вашему файлу с данными

# 2. Предобработка данных
categories = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]

# Унификация текста: приведение к нижнему регистру и удаление лишних символов
data['comment'] = data['comment'].astype(str).str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

labels = data[categories].values.astype(int)

# Стратифицированная выборка по одному из наиболее сбалансированных признаков (если есть)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment'].values,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels.argmax(axis=1)  # предполагается, что есть баланс по классам
)

# 3. Токенизация и подготовка датасета
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.FloatTensor(label)
        }

train_dataset = CommentsDataset(train_texts, train_labels, tokenizer)
val_dataset = CommentsDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 4. Модель с несколькими выходами (multi-label)
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=False)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(categories))
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [batch_size, hidden_size]
        pooled_output = self.dropout(pooled_output)
        logits= self.classifier(pooled_output)  # [batch_size, num_classes]
        return logits

model= BertMultiLabelClassifier()
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 5. Обучение с планировщиком lr и клиппингом градиентов
criterion= nn.BCEWithLogitsLoss()
optimizer= optim.AdamW(model.parameters(), lr=2e-5)


num_epochs=4  # увеличьте число эпох для лучшего обучения

total_steps= len(train_loader)*num_epochs

scheduler= get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1*total_steps),
    num_training_steps=total_steps
)

for epoch in range(num_epochs):
    model.train()
    total_loss=0
    for batch in train_loader:
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels= batch['labels'].to(device)

        optimizer.zero_grad()
        outputs= model(input_ids=input_ids, attention_mask=attention_mask)
        loss= criterion(outputs, labels)
        loss.backward()

        # градиентный клиппинг для стабилизации обучения
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        total_loss+= loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

# Валидация и расчет метрик (F1 и ROC-AUC)
model.eval()
all_preds=[]
all_true=[]

with torch.no_grad():
    for batch in val_loader:
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels=batch['labels'].cpu().numpy()

        outputs= model(input_ids=input_ids, attention_mask=attention_mask)
        preds=torch.sigmoid(outputs).cpu().numpy()

        all_preds.extend(preds)
        all_true.extend(labels)

pred_labels=np.array(all_preds)>=0.5  # порог можно подбирать отдельно

true_labels=np.array(all_true)


Epoch 1/4, Loss: 0.5376
Epoch 2/4, Loss: 0.4133
Epoch 3/4, Loss: 0.3561
Epoch 4/4, Loss: 0.3253


In [14]:
# F1-score macro (по всем классам вместе) и micro (по всем примерам и классам вместе)
f1_macro=f1_score(true_labels,pred_labels,average='macro')
f1_micro=f1_score(true_labels,pred_labels,average='micro')

print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (micro): {f1_micro:.4f}")

true_labels = np.array(all_true)
preds = np.array(all_preds)

# ROC-AUC для каждого класса и средний по классам
for i, category in enumerate(categories):
    try:
        roc_auc = roc_auc_score(true_labels[:, i], preds[:, i])
    except ValueError:
        roc_auc=None  # если только один класс в данных или ошибка при вычислении
    print(f"ROC-AUC для '{category}': {roc_auc if roc_auc is not None else 'недоступен'}")

# Средний ROC-AUC по всем классам (если хотите усреднить по классам)
try:
    roc_auc_mean=np.mean([roc_auc_score(true_labels[:,i], all_preds[:,i]) for i in range(len(categories))])
except:
    roc_auc_mean=None

if roc_auc_mean is not None:
    print(f"Средний ROC-AUC по классам: {roc_auc_mean:.4f}")
else:
    print("Не удалось вычислить средний ROC-AUC.")

F1-score (macro): 0.4526
F1-score (micro): 0.6409
ROC-AUC для 'Вопрос решен': 0.8178879310344828
ROC-AUC для 'Нравится качество выполнения заявки': 0.8481334841628959
ROC-AUC для 'Нравится качество работы сотрудников': 0.9118001930501931
ROC-AUC для 'Нравится скорость отработки заявок': 0.978734639016897
ROC-AUC для 'Понравилось выполнение заявки': 0.5681714546283629
ROC-AUC для 'Другое': 0.8886219698624154
Не удалось вычислить средний ROC-AUC.
